In [275]:
# !pip install pydantic
import pandas as pd
from pydantic import BaseModel, field_validator
from typing import Optional
from datetime import datetime

# Exemplos Simples

## Validando Strings

In [277]:
class ValidaStr(BaseModel):
    input:str
    # Nesse caso Optional é o valor, ele precisa ser passado de alguma forma
    output:Optional[str] = None

input = "Olá"
output = "Tudo bem?"

ValidaStr(input=input)#,output=output)


ValidaStr(input='Olá', output=None)

## Validando Dicionários

In [250]:
from pydantic import BaseModel

class Pessoa(BaseModel):
    nome:str
    idade:int
    # peso:float
    # email: str  
    
    # Não realiza a conversão automática de data type
    class Config:
        strict = True

data = {"nome": "Alice",
    "idade": 30,
    "peso":28.5,
    "email": "alice@example.com"}

validated_data = Pessoa(**data)
print(validated_data)

nome='Alice' idade=30


## Exemplo com a conversão automática

In [251]:
class Pessoa(BaseModel):
    nome:str
    idade:int
    # peso:float
    # email: str  

        # class Config:
        # strict = True

data_2 =  [{"nome": "Alice",
    "idade": 30,
    "peso":28.5,
    "email": "alice@example.com"},
    {"nome": "Maria","idade":"30"}]

for item in data_2:
    try:
        validated_data = Pessoa(**item)
        # print(validated_data)
    except Exception as e:
        print(e)
        print(item)

In [252]:
# Se eu tiver dados gerados como retirar eles e já realizar a validação das informações
Pessoa(**{'nome':'Matheus','idade':5})

Pessoa(nome='Matheus', idade=5)

# Exemplo com Dataframes

## Validação de colunas

In [ ]:
class CountryData(BaseModel):
    
    country_name: str
    country_code: str
    series_name: Optional[str] 
    series_code: Optional[str]

    # Validação para garantir que campos obrigatórios não sejam vazios
    @field_validator("country_name","country_code")
    def not_empty(cls, value):
        if not value or not value.strip():
            raise ValueError("O campo não pode estar vazio")
        return value
    
df = pd.read_csv("../data/sdg_literacy_rate.csv")
df = df.drop(['1990 [YR1990]', '1991 [YR1991]', '1992 [YR1992]', '1993 [YR1993]',
       '1994 [YR1994]', '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]',
       '1998 [YR1998]', '1999 [YR1999]', '2000 [YR2000]', '2001 [YR2001]',
       '2002 [YR2002]', '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]',
       '2006 [YR2006]', '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]',
       '2010 [YR2010]', '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]',
       '2014 [YR2014]', '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]',
       '2018 [YR2018]', '2019 [YR2019]', '2020 [YR2020]'],axis=1)

df.head()


,Country Name,Country Code,Series Name,Series Code
0,World,WLD,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS
1,Afghanistan,AFG,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS
2,Albania,ALB,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS
3,Algeria,DZA,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS
4,American Samoa,ASM,"Literacy rate, youth total (% of people ages 1...",SE.ADT.1524.LT.ZS


In [254]:
def validar_dados(df):
    # Lista para armazenar dados válidos e erros
    validated_data = []
    errors = []

    # Validar cada linha do DataFrame
    for _, row in df.iterrows():
        entry = {
        "country_name":row["Country Name"],
        "country_code":row["Country Code"],
        "series_name":row["Series Name"],
        "series_code":row["Series Code"]
        }

        try: 
            # Validar com o modelo Pydantic
            validated_entry = CountryData(**entry)
            validated_data.append(validated_entry)
        except Exception as e:
            print(f"Erro na linha: {entry}")
            # print(f"Detalhes do erro: {e}")
            errors.append({"entry": entry, "error": str(e)})

    return validated_data, errors

# Exibir resultados
validated_data, errors = validar_dados(df)
print(f"Linhas válidas: {len(validated_data)}")
print(f"Linhas com erro: {len(errors)}")

Erro na linha: {'country_name': nan, 'country_code': nan, 'series_name': nan, 'series_code': nan}
Erro na linha: {'country_name': nan, 'country_code': nan, 'series_name': nan, 'series_code': nan}
Erro na linha: {'country_name': nan, 'country_code': nan, 'series_name': nan, 'series_code': nan}
Erro na linha: {'country_name': 'Data from database: Sustainable Development Goals (SDGs)', 'country_code': nan, 'series_name': nan, 'series_code': nan}
Erro na linha: {'country_name': 'Last Updated: 07/22/2022', 'country_code': nan, 'series_name': nan, 'series_code': nan}
Linhas válidas: 261
Linhas com erro: 5


In [255]:
# Filtrar linhas com valores válidos para as colunas principais
df = df.dropna(subset=["Country Name","Country Code"]).reset_index(drop=True)

# Exibir resultados
validated_data, errors = validar_dados(df)
print(f"Linhas válidas: {len(validated_data)}")
print(f"Linhas com erro: {len(errors)}")

Linhas válidas: 261
Linhas com erro: 0


# Exemplo dataframe 2

In [256]:
df_1 = pd.read_csv("../data/sdg_electricity_data.csv")
df_1.head()

,Series Name,Series Code,Country Name,Country Code,1990 [YR1990],2000 [YR2000],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,Access to electricity (% of population),EG.ELC.ACCS.ZS,Arab World,ARB,..,80.7361412102573,87.3326612400273,87.0395883900211,88.992619805618,88.0153561759997,88.681885670696,89.195062354407,90.3246594905463,88.9107494038454,89.9999455504114,90.2777350826821
1,Access to electricity (% of population),EG.ELC.ACCS.ZS,Caribbean small states,CSS,..,86.8292942974403,93.3906480037973,94.4029957151005,94.8869223835886,94.8017508751529,95.6659287364673,96.6912081788071,97.2997114625456,97.6860301364799,98.221954027314,98.8189268976713
2,Access to electricity (% of population),EG.ELC.ACCS.ZS,Central Europe and the Baltics,CEB,100,100,100,100,100,100,100,99.8278344126722,100,100,100,99.9796400333302
3,Access to electricity (% of population),EG.ELC.ACCS.ZS,Early-demographic dividend,EAR,..,66.7030724363987,75.1718689357295,81.1046083548977,82.2652548788111,83.1131680571248,85.73149882183,87.0192357449593,89.376836771094,91.1103260621346,92.2512760560982,93.3822156743267
4,Access to electricity (% of population),EG.ELC.ACCS.ZS,East Asia & Pacific,EAS,..,92.2236372239929,96.1416216132401,96.3971680759818,96.6026789439967,96.7282517141434,97.0996797638928,97.198108141376,97.8548149745798,97.8737145984247,98.0575567896267,98.0133338488244


In [257]:
class Validando(BaseModel):

    Series_Name: str
    Series_Code: str
    Country_Name: str
    Country_Code: str

def validar_dados_2(df = df_1):
    validado = []
    erro = []

    for _, row in df_1.iterrows():
        entry = {
        "Country_Name":row["Country Name"],
        "Country_Code":row["Country Code"],
        "Series_Name":row["Series Name"],
        "Series_Code":row["Series Code"]
        }
        try:
            data_valid = Validando(**entry)
            validado.append(data_valid)
        except Exception as e:
            # print(entry)
            # print(e)
            erro.append(entry)

    return validado, erro

validado, erro = validar_dados_2()
print(len(validado))
print(len(erro))


261
5


In [258]:
# Utilizado para preencher valores ausentes no df, no exemplo substituir por 0
# df_1_erro = df_1.fillna(0)

# Para encontrar os valores nulos:
df_1_erro = df_1.isnull()
df_1_erro

,Series Name,Series Code,Country Name,Country Code,1990 [YR1990],2000 [YR2000],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
262,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
263,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
264,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [259]:
df_1 = df_1.dropna(subset=["Series Name","Series Code","Country Name","Country Code"]).reset_index(drop=True)
validado, erro = validar_dados_2()
print(len(validado))
print(len(erro))

261
0


In [260]:
validado, erro = validar_dados_2()
print(len(validado))
print(len(erro))

261
0


## Dataframe Titanic

In [261]:
df_titanic = pd.read_csv("../data/titanic.csv")
cols = df_titanic.columns
replace_cols = ["Sobrevivente","Classe","Nome","Sexo","Idade","Esposas a bordo","Filhos a bordo","Tarifa"]
df_titanic[replace_cols] = df_titanic[cols]
df_titanic = df_titanic.drop(columns=cols)

In [265]:
class ValidandoTitanic(BaseModel):
    sobrevivente:int
    classe:int
    nome:str
    sexo:str
    idade:int#float

# Tras os dados e o índice
# for item in df_titanic.iterrows():
#     print(item)#[1]

def validando_titanic(df = df_titanic):
    # Ignora o índice
    for index,row in df_titanic.iterrows():
        # print(row)
        entrada={
        "sobrevivente":row["Sobrevivente"],
        "classe":row["Classe"],
        "nome":row["Nome"],
        "sexo":row["Sexo"],
        "idade":row["Idade"],}
        try:
            dados_validados = ValidandoTitanic(**entrada)
            # print(dados_validados)
        except Exception as e:
            print(e) 
    return dados_validados
            

In [267]:
df_titanic['Idade'] = df_titanic['Idade'].astype(int)
validando_titanic(df_titanic)


ValidandoTitanic(sobrevivente=0, classe=3, nome='Mr. Patrick Dooley', sexo='male', idade=32)